# <font style="color:blue">YOLOv3 Assignment</font>


# <font style="color:blue">1. Link Google Drive</font>
I found it easier to use Google Drive while training and testing, but let me knwow if you prefer a solution that doesn't need it.

### <font style="color:green">Link google drive</font>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/My\ Drive

/content/drive/My Drive


# <font style="color:blue">2. Clone and build DarkNet repository </font>
Buidl darknet from source. These two setps can be skipped if already available

In [3]:
!git clone https://github.com/AlexeyAB/darknet.git

fatal: destination path 'darknet' already exists and is not an empty directory.


# <font style="color:blue">3. Compile DarkNet</font>

In [4]:
%cd darknet

/content/drive/My Drive/darknet


In [5]:
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

print("Building. . . It might take 2-3 minutes")

!make &> build_log.txt

Building. . . It might take 2-3 minutes


## <font style="color:blue">4. Download Data</font>
Download facemask dataset.

In [6]:
!wget "https://www.dropbox.com/s/uq0x32w70c390fb/mask_no-mask_dataset.zip?dl=1" -O OID-data.zip
!mkdir -p mask_no-mask_dataset
!unzip -o OID-data.zip -d mask_no-mask_dataset

--2022-09-18 15:16:31--  https://www.dropbox.com/s/uq0x32w70c390fb/mask_no-mask_dataset.zip?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.67.18, 2620:100:6031:18::a27d:5112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.67.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/uq0x32w70c390fb/mask_no-mask_dataset.zip [following]
--2022-09-18 15:16:32--  https://www.dropbox.com/s/dl/uq0x32w70c390fb/mask_no-mask_dataset.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uca695459840d2c44258f4839ef3.dl.dropboxusercontent.com/cd/0/get/BtIPf6LQUAUOfHZj5-eVqRZSpEGdqHWO4wC5gyckoAuC3B9LerOl5hjLqaAEczYHp32FRMWY3-xKJNj8GMu_I0ORF7DHo40abQ-dSBt7KK2HWuAS2f67al1Z9AH4N7HzXAqwEQs01Q7dvw1F3QF5gb7OYQ_EJzLbwVVXVda-6Uk2MJ6Iy0MhLCpg9WHKiNHUBLw/file?dl=1# [following]
--2022-09-18 15:16:32--  https://uca695459840d2c44258f4839ef3.dl.dropboxusercontent.com/cd/0/get/BtIPf6LQUAUOfHZj5-

## <font style="color:blue">5. Prepare dataset files</font>
Generate data_train.txt and data_test.txt

In [7]:
import random
import os
import subprocess
import sys

image_dir = "./mask_no-mask_dataset"
f_val = open("data_test.txt", 'w')
f_train = open("data_train.txt", 'w')

path, dirs, files = next(os.walk(image_dir))
data_size = len(files)

ind = 0
data_test_size = int(0.2 * data_size)
test_array = random.sample(range(data_size), k=data_test_size)

for f in os.listdir(image_dir):
    if(f.split(".")[-1] == "jpg"):
        ind += 1
        
        if ind in test_array:
            f_val.write(image_dir+'/'+f+'\n')
        else:
            f_train.write(image_dir+'/'+f+'\n')

f_train.close()
f_val.close()

# <font style="color:blue">6. Download config files</font>
This will download the class names and config files I did modify for training with the facemask dataset

In [8]:
!gdown "1vIGuTDIPbxGpInYDhtc44iDmGQkcxel2"
!gdown "1vGYLcxBzaYV93f4UDMih7ZC55XwoUAkG"
!gdown "1zjpmsB1i-yt99PLvp70AWHu-O2Pe_fTl"
!gdown "1v98eWlEhhTIVGMb8Cbxw7advaVXryXmB"

Downloading...
From: https://drive.google.com/uc?id=1vIGuTDIPbxGpInYDhtc44iDmGQkcxel2
To: /content/drive/MyDrive/darknet/class.names
100% 11.0/11.0 [00:00<00:00, 21.5kB/s]
Downloading...
From: https://drive.google.com/uc?id=1vGYLcxBzaYV93f4UDMih7ZC55XwoUAkG
To: /content/drive/MyDrive/darknet/yolov3-facemask-test.cfg
100% 8.45k/8.45k [00:00<00:00, 15.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1zjpmsB1i-yt99PLvp70AWHu-O2Pe_fTl
To: /content/drive/MyDrive/darknet/yolov3-facemask-train.cfg
100% 8.45k/8.45k [00:00<00:00, 16.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1v98eWlEhhTIVGMb8Cbxw7advaVXryXmB
To: /content/drive/MyDrive/darknet/yolov3-facemask-setup.data
100% 95.0/95.0 [00:00<00:00, 164kB/s]


You can see there are 4 files which have been downloaded. We will go through each file in the next section.

# <font style="color:blue">7. Download weights for Convolutional backbone</font>
Download pre-trained model.

In [9]:
!wget "https://www.dropbox.com/s/18dwbfth7prbf0h/darknet53.conv.74?dl=1" -O darknet53.conv.74

--2022-09-18 15:17:27--  https://www.dropbox.com/s/18dwbfth7prbf0h/darknet53.conv.74?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6031:18::a27d:5112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/18dwbfth7prbf0h/darknet53.conv.74 [following]
--2022-09-18 15:17:27--  https://www.dropbox.com/s/dl/18dwbfth7prbf0h/darknet53.conv.74
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc9ab96a49bf1f38432c6abb0458.dl.dropboxusercontent.com/cd/0/get/BtIUXjQxc9MyUWUbyMBYv5sOI2bxKEgCwp3scqYaTTU31Rm-YDyPhX-F4PB9tjg58AgIxetmCElyzhakkUtb4SUardfD4BXYyB40kCjI5MCZsJ_7EhTJi0s0CFsNnRoAug4KCqRr17j1Qtg_1fgdVygVPVw1ir7gZMyMzI_bnUhBjwzCYwpgEX2scQBcGg7O9I0/file?dl=1# [following]
--2022-09-18 15:17:28--  https://uc9ab96a49bf1f38432c6abb0458.dl.dropboxusercontent.com/cd/0/get/BtIUXjQxc9MyUWUbyMBYv5sOI2bxKEgCwp3scqY

# <font style="color:blue">8. Start Training </font>

In [11]:
!./darknet detector train yolov3-facemask-setup.data yolov3-facemask-train.cfg ./darknet53.conv.74 -dont_show -map -clear

 CUDA-version: 11010 (11020), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
 Prepare additional network for mAP calculation...
 0 : compute_capability = 750, cudnn_half = 0, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 32, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    608 x 608 x   3 ->  608 x 608 x  32 0.639 BF
   1 conv     64       3 x 3/ 2    608 x 608 x  32 ->  304 x 304 x  64 3.407 BF
   2 conv     32       1 x 1/ 1    304 x 304 x  64 ->  304 x 304 x  32 0.379 BF
   3 conv     64       3 x 3/ 1    304 x 304 x  32 ->  304 x 304 x  64 3.407 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 304 x 304 x  64 0.006 BF
   5 conv    128       3 x 3/ 2    304 x 304 x  64 ->  152 x 152 x 128 3.407 BF
   6 conv     64       1 x 1/ 1    152 x 152 x 128 ->  152 x 152 x  64 0.379 BF
   7 conv    128       3 x 3/ 1    152 x 152 x  64 ->